<a href="https://colab.research.google.com/github/ezinneanne/Farmeye/blob/new_branch/plantdisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [4]:
# Set parameters
IMG_SIZE = (224, 224)  # Resize images to 224x224
BATCH_SIZE = 32
DATA_DIR = '/content/drive/MyDrive/image_dataset'  # Path to the dataset
OUTPUT_DIR = '/content/split_data'  # Directory for split data - Colab's local storage
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Step 1: Organize data into train and validation directories manually
# def split_data(data_dir, output_dir, val_size=0.2):
#     # Create necessary directories for training and validation sets
#     train_dir = os.path.join(output_dir, 'train')
#     val_dir = os.path.join(output_dir, 'val')
#     if not os.path.exists(train_dir):
#         os.makedirs(train_dir)
#     if not os.path.exists(val_dir):
#         os.makedirs(val_dir)

#     # Iterate through each class folder
#     for class_name in os.listdir(data_dir):
#         class_folder = os.path.join(data_dir, class_name)

#         if os.path.isdir(class_folder):  # If it's a folder (class)
#             # Create subfolders in train and validation directories
#             os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
#             os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

#             # Get all image file paths
#             image_paths = [os.path.join(class_folder, f) for f in os.listdir(class_folder)]

#             # Split data into train and validation sets
#             train_paths, val_paths = train_test_split(image_paths, test_size=val_size, random_state=42)

#             # Copy files to respective directories instead of moving them
#             for path in train_paths:
#                 shutil.copy(path, os.path.join(train_dir, class_name, os.path.basename(path)))
#             for path in val_paths:
#                 shutil.copy(path, os.path.join(val_dir, class_name, os.path.basename(path)))

# split_data(DATA_DIR, OUTPUT_DIR)

In [ ]:
# Step 2: Create ImageDataGenerators for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize pixel values to [0,1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Step 3: Create data generators for training and validation sets
train_generator = train_datagen.flow_from_directory(
    os.path.join(OUTPUT_DIR, 'train'),  # Updated with the split data path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # 'categorical' because we have multiple classes
)

validation_generator = validation_datagen.flow_from_directory(
    os.path.join(OUTPUT_DIR, 'val'),  # Updated with the split data path
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
# Step 4: Build the Model using EfficientNetB0 for transfer learning
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze the base model layers

# Build the model on top of EfficientNetB0
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(22, activation='softmax')  # 30 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 5: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Step 6: Save the model
model.save('crop_pest_disease_classifier.keras')